In [21]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone

In [23]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)


In [9]:
with open("../Nvidia_2024_Quarterly_Results/NVIDIA_1st_quarter_2024.txt") as fh:
    tt  = fh.read()
    splits = text_splitter.split_text(tt)
    doc = text_splitter.create_documents(splits) #this should return the list of documents.

In [17]:
len(doc)

57

In [13]:
doc[0].page_content

'NVIDIA Corporation - NVIDIA Announces Financial Results for First Quarter Fiscal 2024\nSkip to main content\nNewsEvents & Presentations\nEvents & PresentationsPresentations\nStock Info\nStock Quote & ChartHistorical Price LookupInvestment CalculatorFundamentalsAnalyst Coverage\nFinancial Info\nFinancial ReportsSEC FilingsQuarterly ResultsAnnual Reports and ProxiesAnnual Meeting\nGovernance\nManagement TeamBoard of DirectorsGovernance DocumentsCommittee CompositionContact the Board\nInvestor Resources'

In [14]:
embedding = embeddings.embed_documents([doc[0].page_content])

In [20]:
def read_file(filename):
  fh = open(filename, "r")
  try:
      return fh.read()
  finally:
      fh.close()

In [24]:
pc = Pinecone()

print("Connecting to Pinecone index")
index_name = 'chatbot-example-nvidia-quarter'
index = pc.Index(index_name)
index.describe_index_stats()

# text_field = "text"
# # embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
# # # vectorstore = PineconeVectorStore(index, embeddings, text_field)

Connecting to Pinecone index


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [25]:
directory = "../Nvidia_2024_Quarterly_Results"
for filename in os.listdir(directory):
        if filename.endswith(('.txt', '.docx', '.pdf', '.csv')):
            file_path = os.path.join(directory, filename)
            content = read_file(file_path)
            splits = text_splitter.split_text(content)
            doc = text_splitter.create_documents(splits) #this should return the list of documents.
            for i in range(len(doc)):
                embedding = embeddings.embed_documents([doc[i].page_content])[0]
                
                meta = {'text': doc[i].page_content}

                index.upsert(vectors=[(f"{filename}_doc{i}", embedding, meta)])
                